In [1]:
import numpy as np
import math

In [2]:

def value2bin(data,res,quan_bit):
  binary_string=bin(data)[2:].zfill(quan_bit)
  bin_arr = np.array([int(x) for x in binary_string])
  temp = bin_arr.reshape(-1,res)
  res_list=[]
  for i in temp:
    res = int("".join(str(x) for x in i), 2)
    res_list.append(res)
  return res_list

def test_compress_tc(tc_arr,ou_row,ou_col,res,quan_bit):
  tr_data = tc_arr.transpose()
  rram_r=[]
  total_rram=[]
  for row in tr_data:
    rram_r=[]
    for e in row:
      temp = value2bin(e,res,quan_bit)

      rram_r.extend(temp)
    total_rram.append(rram_r)

  total_tr = np.array(total_rram).transpose()
  tmp_a=[]
  for t in total_tr:
    a = int("".join(str(x) for x in t), 2)
    tmp_a.append(a)
  #xbar

  uni_temp = set(tmp_a)
  #print(uni_temp)
  uni_temp.discard(0)

  a_idx=[]
  for i in uni_temp:
    l=[]
    for idx, v in enumerate(tmp_a):
      if(i==v & v!=0):
        l.append(idx)
    if(l):
      a_idx.append(l)

  #print(tmp_a)
  #print(a_idx)
  #print(uni_temp)

  return a_idx,uni_temp

def test_tc_preprocess(tc_arr,ou_row,ou_col,res,quan_bit,m_rank,n_rank):
  #tc_s=tc_arr.shape
  tc_m, tc_n = math.ceil(tc_arr.shape[0]/m_rank),math.ceil(tc_arr.shape[1]/n_rank)
  r_time = math.ceil(tc_arr.shape[1]/ou_row)
  #tc_tr = tc_arr.transpose()
  #p = math.ceil(quan_bit/res)
  s = 0
  r_s = 0
  total_pat=[]
  total_idx=[]
  for mr in range(tc_m):
    r_s = 0
    ou_pattern0=[]
    ou_idx0=[]
    for r in range(r_time):
      t0=tc_arr[s:s+m_rank,r_s:r_s+ou_row]
      t_idx, uni_t=test_compress_tc(t0,ou_row,ou_col,res,quan_bit)
      #print(t0)
      ou_idx0.append(t_idx)
      ou_pattern0.append(list(uni_t))
      r_s = r_s + ou_row
    s = s + m_rank
    total_pat.append(ou_pattern0)
    total_idx.append(ou_idx0)
  #print('--------')
  #print(total_pat)
  #print(total_idx)
  return total_pat, total_idx

In [3]:
def seperate_posneg(arr):
  arr_pos = (arr>0)*arr
  arr_neg = (arr<0)*arr*(-1)
  return arr_pos, arr_neg



In [4]:
res=1
quan_bit=8
m_rank=2
n_rank=3
ou_row=3
ou_col=3
#ipt1 = np.array([1,1,0,1])
#w1 = np.array([[120,10,164,8],[103,212,9,18],[112,11,90,181],[100,9,77,15],[102,101,19,11],[10,99,7,15]])
w1 = np.array([[1,2,3],[0,0,0],[-1,-2,-3],[4,5,6],[0,0,0],[-4,-5,-6]])
pos_w1, neg_w1 = seperate_posneg(w1)

print(pos_w1)
print(neg_w1)
pos_w1_pat, pos_w1_idx=test_tc_preprocess(pos_w1,ou_row,ou_col,res,quan_bit,m_rank,n_rank)

neg_w1_pat, neg_w1_idx=test_tc_preprocess(neg_w1,ou_row,ou_col,res,quan_bit,m_rank,n_rank)


#w1_pat, w1_idx=test_tc_preprocess(w1,ou_row,ou_col,res,quan_bit,m_rank,n_rank)


#print(w1_pat)
#print(w1_idx)
#print('~~~~~~~')
#print(len(w1_pat))
#print(len(w1_idx))

[[1 2 3]
 [0 0 0]
 [0 0 0]
 [4 5 6]
 [0 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [1 2 3]
 [0 0 0]
 [0 0 0]
 [4 5 6]]


In [5]:
def do_mac_op(ipt,ou_pattern,ou_idx,m_rank,ou_row,quan_bit):
  rank_reg=[]
  addpartial_reg=[]
  acc_reg=[]

  #our_num = 2
  #our_idx_list=[]
  #for i in range(our_num):
  #  our_idx_list.append(i)
  #print(ou_pattern)
  #print(ou_idx)

  for val,idx in zip(ou_pattern,ou_idx):
    #print(val,'--',idx)
    rank_reg=[]
    for j in range(m_rank): #2之後要參數化
      rank_reg.append(0)
    temp = value2bin(val,1,ou_row) #2之後要參數化
    #print('--',i_v)
    rt = np.matmul(temp,ipt)
    #print(rt)
    #print('idx, ',idx)
    for r0 in idx:
      r_i = math.ceil((r0+1)/quan_bit)-1 #4之後要參數化
      r_s = (quan_bit-1)-math.ceil(r0%quan_bit) #4之後要參數化
      rank_reg[r_i]=rank_reg[r_i] + rt*(2**r_s)
      #print(r0,'..',r_i,'**',r_s)
    #print(rank_reg)
    addpartial_reg.append(rank_reg)
  #print(np.sum(np.array(addpartial_reg), axis = 0))
  acc_reg.append(np.sum(np.array(addpartial_reg), axis = 0))


  return np.sum(np.array(acc_reg), axis = 0)

#def vmm_op2(input_data,tc_pat,tc_idx,md,m_rank,nd,n_rank,ou_row,ou_col,quan_bit):
def vmm_op2(input_data,tc_pat,tc_idx,m_rank,n_rank,ou_row,ou_col,quan_bit):

  rank_reg=[]
  ans=[]

  #ipt0 = np.array([1,1,0,1])

  #for m_i in range(0,4,2): #(0,m(d)*r(d-1),r(d-1))
  #for m_i in range(0,4,4): #(0,m(d)*r(d-1),r(d-1))
  for pat_r,idx_r in zip(tc_pat,tc_idx): #(0,m(d)*r(d-1),r(d-1))
    rank_reg=[]
    for j in range(m_rank): #2之後要參數化 r(d-1)
      rank_reg.append(0)
    #for input_idx in range(0,4,2): #(0,input pattern's length/n(d)*r(d),OU_row)
    pat_idx=0
    #print('pat_r',pat_r)
    #print('idx_r',idx_r)
    #print(input_data)
    #print(ou_row)
    for input_idx in range(0,len(input_data),ou_row): #(0,input pattern's length/n(d)*r(d),OU_row)
      #ou_pattern2, ou_idx2=tc_preprocess(w1[m_i:m_i+2,input_idx:input_idx+2],2,2,1,4,2,2)
      #ou_pattern2, ou_idx2=tc_preprocess(tc_data[m_i:m_i+m_rank,input_idx:input_idx+ou_row],2,2,1,4,4,2)
      #t_idx, uni_t=compress_tc(tc_data[m_i:m_i+m_rank,input_idx:input_idx+ou_row],ou_row,ou_col,1,4)
      # above 4:r(d-1) 2:OU_row
      #pat_list = list(ou_pattern2[0])
      #pat_list = list(uni_t)
      #print('***',pat_list)
      #print(t_idx)
      #print('***',ou_idx2)
      #ou_col_num = math.ceil(len(pat_list)/ou_col)
      input_t0 = input_data[input_idx:input_idx+ou_row]
      pat_list=pat_r[pat_idx]
      t_idx = idx_r[pat_idx]
      #print('pat_list',pat_list)
      #print('t_idx',t_idx)
      # above 2:OU_row
      #partial_ans = do_mac([input_t0],[pat_list[start_ou:start_ou+ou_col]],[ou_idx1[k][start_ou:start_ou+ou_col]],2,2,4)
      start_ou=0
      for i in range(0,len(pat_list),ou_col): #(0,OU內的Pattern個數,OU_col)
        #print('//',pat_list[i:i+2])
        #print('++',ou_idx2[0][i:i+2])

        #partial_ans = do_mac([input_t0],[pat_list[i:i+2]],[ou_idx2[0][i:i+2]],2,2,4)
        #partial_ans = do_mac([input_t0],[pat_list[i:i+2]],[ou_idx2[0][i:i+2]],4,2,4)
        #partial_ans = do_mac_op(input_t0,pat_list[i:i+ou_col],ou_idx2[0][i:i+ou_col],m_rank,2,4)
        partial_ans = do_mac_op(input_t0,pat_list[i:i+ou_col],t_idx[i:i+ou_col],m_rank,ou_row,quan_bit)
        # above 2 OU_col
        #print('partial ans',partial_ans)
        rank_reg = rank_reg + partial_ans
        #start_ou=start_ou+ou_col
      pat_idx=pat_idx+1
    ans.append(rank_reg)
  #print(ans)
  return ans


def input_compress(input_data,res,quan_bit):
  tr_data = input_data.transpose()
  rram_r=[]
  total_rram=[]
  #print(tr_data)
  for row in tr_data:
    rram_r=[]
    for e in row:
      temp = value2bin(e,res,quan_bit)

      #print(temp)
      rram_r.extend(temp)
    total_rram.append(rram_r)
    total_tr = np.array(total_rram).transpose()
  tmp_a=[]
  for t in total_tr:
    a = int("".join(str(x) for x in t), 2)
    tmp_a.append(a)
  #xbar

  uni_temp = set(tmp_a)
  #print(uni_temp)
  uni_temp.discard(0)

  a_idx=[]
  for i in uni_temp:
    l=[]
    for idx, v in enumerate(tmp_a):
      if(i==v & v!=0):
        l.append(quan_bit-1-idx)
    if(l):
      a_idx.append(l)

  #print(tmp_a)
  #print(a_idx)
  #print(uni_temp)
  return list(uni_temp), a_idx

In [6]:
'''
m_r_cnt=0
cycle=0
input_idx_s=0
rank_reg0=[]
test_input0=[1,1,0,1]
pat_idx=0
pat_list_i=0
ans0=[]
for j in range(m_rank): #2之後要參數化 r(d-1)
  rank_reg0.append(0)
while(1):
  cycle = cycle + 1
  pat_r = w1_pat[m_r_cnt]
  idx_r = w1_idx[m_r_cnt]


  input_t0 = test_input0[input_idx_s:input_idx_s+ou_row]
  print(input_t0)
  print(pat_idx)
  print(pat_r[pat_idx])
  pat_list=pat_r[pat_idx]
  t_idx = idx_r[pat_idx]

  partial_ans = do_mac_op(input_t0,pat_list[pat_list_i:pat_list_i+ou_col],t_idx[pat_list_i:pat_list_i+ou_col],m_rank,ou_row,quan_bit)
  # above 2 OU_col
  #print('partial ans',partial_ans)
  rank_reg0 = rank_reg0 + partial_ans
  if(pat_list_i >= len(pat_list)-1):
    pat_list_i=0
    pat_idx = pat_idx + 1
    if(input_idx_s >= len(test_input0)-1):
      input_idx_s = 0
      pat_idx = 0
      ans0.append(rank_reg0)
      rank_reg0=[]
      for j in range(m_rank):
        rank_reg0.append(0)
      if(m_r_cnt==len(w1_pat)-1):
        break
      else:
        m_r_cnt = m_r_cnt + 1
    else:
      input_idx_s = input_idx_s + ou_row

  else:
    pat_list_i = pat_list_i + ou_col


print(ans0)

'''
#---------------------------
'''
def vmm_op2(input_data,tc_pat,tc_idx,md,m_rank,nd,n_rank,ou_row,ou_col,quan_bit):
  rank_reg=[]
  ans=[]
  for pat_r,idx_r in zip(tc_pat,tc_idx): #(0,m(d)*r(d-1),r(d-1))
    rank_reg=[]
    for j in range(m_rank): #2之後要參數化 r(d-1)
      rank_reg.append(0)
    pat_idx=0

    for input_idx in range(0,len(input_data),ou_row): #(0,input pattern's length/n(d)*r(d),OU_row)

      input_t0 = input_data[input_idx:input_idx+ou_row]
      pat_list=pat_r[pat_idx]
      t_idx = idx_r[pat_idx]

      for i in range(0,len(pat_list),ou_col):


        partial_ans = do_mac_op(input_t0,pat_list[i:i+ou_col],t_idx[i:i+ou_col],m_rank,ou_row,quan_bit)

        rank_reg = rank_reg + partial_ans

      pat_idx=pat_idx+1
    ans.append(rank_reg)

  return ans
'''


"\ndef vmm_op2(input_data,tc_pat,tc_idx,md,m_rank,nd,n_rank,ou_row,ou_col,quan_bit):\n  rank_reg=[]\n  ans=[]\n  for pat_r,idx_r in zip(tc_pat,tc_idx): #(0,m(d)*r(d-1),r(d-1))\n    rank_reg=[]\n    for j in range(m_rank): #2之後要參數化 r(d-1)\n      rank_reg.append(0)\n    pat_idx=0\n\n    for input_idx in range(0,len(input_data),ou_row): #(0,input pattern's length/n(d)*r(d),OU_row)\n\n      input_t0 = input_data[input_idx:input_idx+ou_row]\n      pat_list=pat_r[pat_idx]\n      t_idx = idx_r[pat_idx]\n\n      for i in range(0,len(pat_list),ou_col):\n\n\n        partial_ans = do_mac_op(input_t0,pat_list[i:i+ou_col],t_idx[i:i+ou_col],m_rank,ou_row,quan_bit)\n\n        rank_reg = rank_reg + partial_ans\n\n      pat_idx=pat_idx+1\n    ans.append(rank_reg)\n\n  return ans\n"

In [7]:
#inp_2d = np.array([[16,19,108],[109,150,109],[161,20,110],[106,8,111]])
inp_2d = np.array([[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18]])
inp_2d_tr = inp_2d.transpose()

#res_list=[]
val=[]

for input_val0 in inp_2d_tr:
  #input_pattern, input_idx=input_compress(np.array([[8,7,6,5]]))
  #input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)
  input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)
  #print(input_pattern)
  #print(input_idx)
  res_list0=[]
  res_list1=[]
  #res=[0,0,0,0]
  for l in range(len(w1)):
    res_list0.append(0)
    res_list1.append(0)

  for ipt_val, ipt_idx in zip(input_pattern, input_idx):
    ipt0=value2bin(ipt_val,res,w1.shape[1])
    #print(ipt0)
    test_a0 = vmm_op2(ipt0,pos_w1_pat,pos_w1_idx,m_rank,n_rank,ou_row,ou_col,quan_bit)
    test_a1 = vmm_op2(ipt0,neg_w1_pat,neg_w1_idx,m_rank,n_rank,ou_row,ou_col,quan_bit)

    #print('test_a',test_a)
    temp_a0 = np.array(test_a0).reshape(1,-1)
    temp_a1 = np.array(test_a1).reshape(1,-1)

    for i_idx in ipt_idx:
      res_list0 = res_list0 + temp_a0[0]*(2**i_idx)
      res_list1 = res_list1 + temp_a1[0]*(2**i_idx)

  print('result0',res_list0)
  print('result1',res_list1*-1)
  val.append(res_list0+(-1)*res_list1)

#print(np.array(val).transpose())



result0 [ 54   0   0 117   0   0]
result1 [   0    0  -54    0    0 -117]
result0 [ 60   0   0 132   0   0]
result1 [   0    0  -60    0    0 -132]
result0 [ 66   0   0 147   0   0]
result1 [   0    0  -66    0    0 -147]
result0 [ 72   0   0 162   0   0]
result1 [   0    0  -72    0    0 -162]
result0 [ 78   0   0 177   0   0]
result1 [   0    0  -78    0    0 -177]
result0 [ 84   0   0 192   0   0]
result1 [   0    0  -84    0    0 -192]


In [8]:
#print(val)

val3to2 = np.array(val).transpose()
print(val3to2)

[[  54   60   66   72   78   84]
 [   0    0    0    0    0    0]
 [ -54  -60  -66  -72  -78  -84]
 [ 117  132  147  162  177  192]
 [   0    0    0    0    0    0]
 [-117 -132 -147 -162 -177 -192]]


In [9]:
def transform(arr,m_list,n_list,h,r):
  arr_tr = np.transpose(arr)
  arr_re = arr_tr.reshape(n_list[h-1],-1)
  new_arr_width = len(arr_re[0])
  new_arr=[]
  p = new_arr_width
  for i in range(0,p,r):
    temp = arr_re[:,i:i+r]
    new_arr.append(temp.reshape(-1))

  return np.transpose(new_arr)

m_list=[2,1,2]
n_list=[2,3,3]
r = 3
#v1_out = [[5,6,8,9],[0,0,1,0],[1,2,3,4]]
v2_input = transform(val3to2,m_list,n_list,2,r)
print(v2_input)

[[  54  117   60  132]
 [   0    0    0    0]
 [ -54 -117  -60 -132]
 [  66  147   72  162]
 [   0    0    0    0]
 [ -66 -147  -72 -162]
 [  78  177   84  192]
 [   0    0    0    0]
 [ -78 -177  -84 -192]]


In [10]:
tc2 = np.array([[1,2,3,4,5,6,7,8,9],[0,1,-4,7,-1,0,-2,-1,-3],[-1,-2,0,-2,0,0,0,2,0]])
print(tc2)

pos_tc2, neg_tc2 = seperate_posneg(tc2)

print(pos_tc2)
print(neg_tc2)
pos_tc2_pat, pos_tc2_idx=test_tc_preprocess(pos_tc2,ou_row,ou_col,res,quan_bit,1,3)

neg_tc2_pat, neg_tc2_idx=test_tc_preprocess(neg_tc2,ou_row,ou_col,res,quan_bit,1,3)

[[ 1  2  3  4  5  6  7  8  9]
 [ 0  1 -4  7 -1  0 -2 -1 -3]
 [-1 -2  0 -2  0  0  0  2  0]]
[[1 2 3 4 5 6 7 8 9]
 [0 1 0 7 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 4 0 1 0 2 1 3]
 [1 2 0 2 0 0 0 0 0]]


In [11]:
v2_tr = v2_input.transpose()

pos_v2_tr, neg_v2_tr = seperate_posneg(v2_tr)

print(pos_v2_tr)
print(neg_v2_tr)

#res_list=[]
s2_val=[]

for input_val0 in pos_v2_tr:
  #input_pattern, input_idx=input_compress(np.array([[8,7,6,5]]))
  #input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)
  input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)
  #print(input_pattern)
  #print(input_idx)
  res_list0=[]
  res_list1=[]
  #res=[0,0,0,0]
  for l in range(len(tc2)):
    res_list0.append(0)
    res_list1.append(0)

  for ipt_val, ipt_idx in zip(input_pattern, input_idx):
    ipt0=value2bin(ipt_val,res,tc2.shape[1])#...w1/tc2
    #print(ipt0)
    test_a0 = vmm_op2(ipt0,pos_tc2_pat,pos_tc2_idx,1,3,ou_row,ou_col,quan_bit)
    test_a1 = vmm_op2(ipt0,neg_tc2_pat,neg_tc2_idx,1,3,ou_row,ou_col,quan_bit)

    #print('test_a',test_a)
    temp_a0 = np.array(test_a0).reshape(1,-1)
    temp_a1 = np.array(test_a1).reshape(1,-1)

    for i_idx in ipt_idx:
      res_list0 = res_list0 + temp_a0[0]*(2**i_idx)
      res_list1 = res_list1 + temp_a1[0]*(2**i_idx)

  print('result0',res_list0)
  print('result1',res_list1*-1)
  s2_val.append(res_list0+(-1)*res_list1)

#print(np.array(s2_val).transpose())
pos_val3 = np.array(s2_val).transpose()
print(pos_val3)

[[ 54   0   0  66   0   0  78   0   0]
 [117   0   0 147   0   0 177   0   0]
 [ 60   0   0  72   0   0  84   0   0]
 [132   0   0 162   0   0 192   0   0]]
[[  0   0  54   0   0  66   0   0  78]
 [  0   0 117   0   0 147   0   0 177]
 [  0   0  60   0   0  72   0   0  84]
 [  0   0 132   0   0 162   0   0 192]]
result0 [864 462   0]
result1 [   0 -156 -186]
result0 [1944 1029    0]
result1 [   0 -354 -411]
result0 [936 504   0]
result1 [   0 -168 -204]
result0 [2124 1134    0]
result1 [   0 -384 -456]
[[ 864 1944  936 2124]
 [ 306  675  336  750]
 [-186 -411 -204 -456]]


In [12]:
v2_tr = v2_input.transpose()

pos_v2_tr, neg_v2_tr = seperate_posneg(v2_tr)

print(pos_v2_tr)
print(neg_v2_tr)

#res_list=[]
s2_val=[]

for input_val0 in neg_v2_tr:
  #input_pattern, input_idx=input_compress(np.array([[8,7,6,5]]))
  #input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)
  input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)
  #print(input_pattern)
  #print(input_idx)
  res_list0=[]
  res_list1=[]
  #res=[0,0,0,0]
  for l in range(len(tc2)):
    res_list0.append(0)
    res_list1.append(0)

  for ipt_val, ipt_idx in zip(input_pattern, input_idx):
    ipt0=value2bin(ipt_val,res,tc2.shape[1])#...w1/tc2
    #print(ipt0)
    test_a0 = vmm_op2(ipt0,pos_tc2_pat,pos_tc2_idx,1,3,ou_row,ou_col,quan_bit)
    test_a1 = vmm_op2(ipt0,neg_tc2_pat,neg_tc2_idx,1,3,ou_row,ou_col,quan_bit)

    #print('test_a',test_a)
    temp_a0 = np.array(test_a0).reshape(1,-1)
    temp_a1 = np.array(test_a1).reshape(1,-1)

    for i_idx in ipt_idx:
      res_list0 = res_list0 + temp_a0[0]*(2**i_idx)
      res_list1 = res_list1 + temp_a1[0]*(2**i_idx)

  print('result0',res_list0)
  print('result1',res_list1*-1)
  s2_val.append(res_list0+(-1)*res_list1)

#print(np.array(s2_val).transpose())
neg_val3 = np.array(s2_val).transpose()
print(neg_val3)

[[ 54   0   0  66   0   0  78   0   0]
 [117   0   0 147   0   0 177   0   0]
 [ 60   0   0  72   0   0  84   0   0]
 [132   0   0 162   0   0 192   0   0]]
[[  0   0  54   0   0  66   0   0  78]
 [  0   0 117   0   0 147   0   0 177]
 [  0   0  60   0   0  72   0   0  84]
 [  0   0 132   0   0 162   0   0 192]]
result0 [1260    0    0]
result1 [   0 -450    0]
result0 [2826    0    0]
result1 [   0 -999    0]
result0 [1368    0    0]
result1 [   0 -492    0]
result0 [3096    0    0]
result1 [    0 -1104     0]
[[ 1260  2826  1368  3096]
 [ -450  -999  -492 -1104]
 [    0     0     0     0]]


In [13]:
for input_val0 in neg_v2_tr:
  print(input_val0)

[ 0  0 54  0  0 66  0  0 78]
[  0   0 117   0   0 147   0   0 177]
[ 0  0 60  0  0 72  0  0 84]
[  0   0 132   0   0 162   0   0 192]


In [14]:
def get_quanbis(arr):
  bits=0
  #print('ll',arr)
  for l0 in arr:
    for l1 in l0:
      #print(l1)
      t = int(l1).bit_length()
      if(t>bits):
        bits = t
  return bits

In [15]:
mat_v2to1=pos_val3-neg_val3
print(mat_v2to1)
mat1 = transform(mat_v2to1,m_list,n_list,1,r)
print(mat1)

pos_mat1, neg_mat1 = seperate_posneg(mat1)

print(pos_mat1)
print(neg_mat1)

pos_mat1_bits = get_quanbis(pos_mat1)
neg_mat1_bits = get_quanbis(neg_mat1)

print(pos_mat1_bits)
print(neg_mat1_bits)

#num=1854
#print(num.bit_length())

[[-396 -882 -432 -972]
 [ 756 1674  828 1854]
 [-186 -411 -204 -456]]
[[-396 -882]
 [ 756 1674]
 [-186 -411]
 [-432 -972]
 [ 828 1854]
 [-204 -456]]
[[   0    0]
 [ 756 1674]
 [   0    0]
 [   0    0]
 [ 828 1854]
 [   0    0]]
[[396 882]
 [  0   0]
 [186 411]
 [432 972]
 [  0   0]
 [204 456]]
11
10


In [16]:
def get_vecquanbis(arr):
  bits=0
  #print('ll',arr)
  for l0 in arr:
    t = int(l0).bit_length()
    if(t>bits):
      bits = t
  return bits



In [17]:
tc1 = np.array([[2,1,-1,3,4,-3],[-2,-1,-1,0,0,-1]])
print(tc1)

pos_tc1, neg_tc1 = seperate_posneg(tc1)

print(pos_tc1)
print(neg_tc1)
pos_tc1_pat, pos_tc1_idx=test_tc_preprocess(pos_tc1,ou_row,ou_col,res,quan_bit,2,2)

neg_tc1_pat, neg_tc1_idx=test_tc_preprocess(neg_tc1,ou_row,ou_col,res,quan_bit,2,2)



[[ 2  1 -1  3  4 -3]
 [-2 -1 -1  0  0 -1]]
[[2 1 0 3 4 0]
 [0 0 0 0 0 0]]
[[0 0 1 0 0 3]
 [2 1 1 0 0 1]]


In [18]:
#v2_tr = v2_input.transpose()

#pos_v2_tr, neg_v2_tr = seperate_posneg(v2_tr)

#print(pos_v2_tr)
#print(neg_v2_tr)

#pos_mat1
pos_v3_tr = pos_mat1.transpose()
#res_list=[]
s1_val=[]

for input_val0 in pos_v3_tr:
  #input_pattern, input_idx=input_compress(np.array([[8,7,6,5]]))
  #input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)

  vec_bits = get_vecquanbis(input_val0)

  input_pattern, input_idx=input_compress(np.array([input_val0]),res,vec_bits)
  #print(input_pattern)
  #print(input_idx)
  res_list0=[]
  res_list1=[]
  #res=[0,0,0,0]
  for l in range(len(tc1)):
    res_list0.append(0)
    res_list1.append(0)

  for ipt_val, ipt_idx in zip(input_pattern, input_idx):
    ipt0=value2bin(ipt_val,res,tc1.shape[1])#...w1/tc2
    #print(ipt0)
    test_a0 = vmm_op2(ipt0,pos_tc1_pat,pos_tc1_idx,2,2,ou_row,ou_col,quan_bit)
    test_a1 = vmm_op2(ipt0,neg_tc1_pat,neg_tc1_idx,2,2,ou_row,ou_col,quan_bit)

    #print('test_a',test_a)
    temp_a0 = np.array(test_a0).reshape(1,-1)
    temp_a1 = np.array(test_a1).reshape(1,-1)

    for i_idx in ipt_idx:
      res_list0 = res_list0 + temp_a0[0]*(2**i_idx)
      res_list1 = res_list1 + temp_a1[0]*(2**i_idx)

  print('result0',res_list0)
  print('result1',res_list1*-1)
  s1_val.append(res_list0+(-1)*res_list1)

#print(np.array(s2_val).transpose())
pos_val0 = np.array(s1_val).transpose()
print(pos_val0)

result0 [4068    0]
result1 [   0 -756]
result0 [9090    0]
result1 [    0 -1674]
[[ 4068  9090]
 [ -756 -1674]]


In [19]:
neg_v3_tr = neg_mat1.transpose()
#res_list=[]
s1_val=[]

for input_val0 in neg_v3_tr:
  #input_pattern, input_idx=input_compress(np.array([[8,7,6,5]]))
  #input_pattern, input_idx=input_compress(np.array([input_val0]),res,quan_bit)

  vec_bits = get_vecquanbis(input_val0)

  input_pattern, input_idx=input_compress(np.array([input_val0]),res,vec_bits)
  #print(input_pattern)
  #print(input_idx)
  res_list0=[]
  res_list1=[]
  #res=[0,0,0,0]
  for l in range(len(tc1)):
    res_list0.append(0)
    res_list1.append(0)

  for ipt_val, ipt_idx in zip(input_pattern, input_idx):
    ipt0=value2bin(ipt_val,res,tc1.shape[1])#...w1/tc2
    #print(ipt0)
    test_a0 = vmm_op2(ipt0,pos_tc1_pat,pos_tc1_idx,2,2,ou_row,ou_col,quan_bit)
    test_a1 = vmm_op2(ipt0,neg_tc1_pat,neg_tc1_idx,2,2,ou_row,ou_col,quan_bit)

    #print('test_a',test_a)
    temp_a0 = np.array(test_a0).reshape(1,-1)
    temp_a1 = np.array(test_a1).reshape(1,-1)

    for i_idx in ipt_idx:
      res_list0 = res_list0 + temp_a0[0]*(2**i_idx)
      res_list1 = res_list1 + temp_a1[0]*(2**i_idx)

  print('result0',res_list0)
  print('result1',res_list1*-1)
  s1_val.append(res_list0+(-1)*res_list1)

#print(np.array(s2_val).transpose())
neg_val0 = np.array(s1_val).transpose()
print(neg_val0)

result0 [2088    0]
result1 [ -798 -1182]
result0 [4680    0]
result1 [-1779 -2631]
[[ 1290  2901]
 [-1182 -2631]]


In [20]:
pos_val0-neg_val0

array([[2778, 6189],
       [ 426,  957]])